# Imports

In [1]:
!pip install clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=c3050af0873fe277b6c5116541fee11033350d8304b91eae625a97756f832c9f
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.15.0
    Uninstalling emoji-2.15.0:
      Successfully uninstalled emoji-2.15.0


In [2]:
# Pytorch related
import torch
import torch.nn as nn

# sklearn related
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Huggingface related
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    DataCollatorForLanguageModeling, 
    TrainingArguments, 
    Trainer, 
    AutoConfig, 
    AutoModelForSequenceClassification
)
from huggingface_hub import snapshot_download

# text preprocessing
from cleantext import clean
import re

# miscellanious
import math
import pandas as pd
import numpy as np
import json
import random
from tqdm import tqdm
import os

2025-11-05 16:05:21.251429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762358721.447641      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762358721.511976      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Mental health detection training

In [3]:
ground_truth_df = pd.read_csv('/kaggle/input/erisk25/final-eriskt2-dataset-with-ground-truth/final-eriskt2-dataset-with-ground-truth/shuffled_ground_truth_labels.txt', sep=' ', header=None)
ground_truth_df.columns = ['user', 'is_depressed']
ground_truth_df.head()

,user,is_depressed
0,subject_BAPb4sn,0
1,subject_x12ArlU,0
2,subject_D1lB5nr,0
3,subject_aEK9WJt,0
4,subject_OpiNZAB,0


In [4]:
with open('/kaggle/input/erisk25/final-eriskt2-dataset-with-ground-truth/final-eriskt2-dataset-with-ground-truth/all_combined/subject_01ZzrIT.json', 'r') as f:
    data = json.load(f)

In [5]:
data[:3]

[{'submission': {'user_id': 'subject_01ZzrIT',
   'target': True,
   'type': 'submission',
   'title': 'Create AI Images within seconds with this app',
   'body': 'Unlimited photos... just TRY IT yourself below and see the magic :) \n\n\n\napp link: [https://www.gentube.app/feed?\\_cid=ja,r](https://www.gentube.app/feed?_cid=ja,r)\n\n',
   'created_utc': '2024-09-06 16:36:22 UTC',
   'submission_id': 'M8kutie',
   'parent_id': None},
  'comments': []},
 {'submission': {'user_id': 'subject_x8JJomn',
   'target': False,
   'type': 'submission',
   'title': '- Ongoing Self-Promotion Thread - Promote your projects here!',
   'body': '&#x200B;\n\nhttps://preview.redd.it/36exw5soj8aa1.jpg?width=512&format=pjpg&auto=webp&s=21931adcf20914015b5105de91455c5070d891b6\n\nr/aiArt focus is the enjoyment, sharing, and discussion regarding our Ai Art creations.\n\nWe know you are as excited to show us what you created as we are to see it. \n\nIn an effort to keep the group tidy and to provide a focal 

In [6]:
posts = {}
root_dir = '/kaggle/input/erisk25/final-eriskt2-dataset-with-ground-truth/final-eriskt2-dataset-with-ground-truth/all_combined'
for file in os.listdir(root_dir):
    file_path = os.path.join(root_dir, file)

    # user id
    user_id = file.split('.')[0]

    # reading the json file
    with open(file_path, 'r') as f:
        file_data = json.load(f)

    user_posts = []
    # using the file data
    for post in file_data:
        if post['submission']['target'] == True:
            user_posts.append(post['submission']['body'])

        comments = post['comments']
        for comment in comments:
            if comment['target'] == True:
                user_posts.append(comment['body'])

    posts[user_id] = user_posts

In [7]:
print(len(posts.keys()))

909


In [8]:
def preprocess(user_posts, N=35):
    # filter removed posts
    user_posts = list(filter(lambda x : x != '[removed]', user_posts))
    texts = ' '.join(user_posts)

    # lowercasing + removing punctuations and urls 
    cleaned = clean(texts, lower=True, no_urls=True, no_punct=True, replace_with_url='')

    # removing emojis and emoticons
    EMOJI_PATTERN = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # chinese characters
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642" 
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+",
        flags=re.UNICODE
    )

    EMOTICON_PATTERN = re.compile(r'(:\)|:-\)|:\(|:-\(|:D|:-D|;\)|;-\)|:P|:-P|xD|XD|:O|:-O|<3|:\'\)|:\'\(|🥲|🤣|😂|😭|😅|😆|😄|😃|😁|😢|😔|😞|😩|😡|😠|😱|😨|😰|😳|😬|😐|😑|😶|😴|🤔|🤗|🤨|🙃|🙂|🤩|🤪|🤯|🤬|🤮|🤢|🤡|🤠|🥳|🥺|🥶|🥵|🤧|🤒|🤕|🤑)', flags=re.IGNORECASE)

    
    cleaned = EMOJI_PATTERN.sub('', cleaned)
    cleaned = EMOTICON_PATTERN.sub('', cleaned)

    # removing hashtags
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    if len(cleaned) == 0:
        return []

    # convert into 35 segments
    segment_size = math.ceil(len(cleaned) / N)
    segments = [cleaned[i : i+segment_size] for i in range(0, len(cleaned), segment_size)]

    if len(segments) < N:
        segments += [""] * (N - len(segments))
    elif len(segments) > N:
        segments = segments[:N]

    return segments

In [9]:
processed_posts = {}
for user_id, user_post in posts.items():
    processed_posts[user_id] = preprocess(user_post)

len(processed_posts)

909

In [10]:
for user_id in processed_posts.keys():
    print(processed_posts[user_id])
    print(len(processed_posts[user_id]))
    break

['ill start watching it curious to see what they got heya so ive been an elemental hero fan since i can remember own the same deck as on master duel now that ive completed it i would like to venture into some', ' new fun themes and possibly find a new favorite deck ive seen dinomorphia and really like the design of the cards but it seems like they dont got a lot going on i have not watched the anime after gx but di', 'd play a bit with tuners is there any anime i should watch that has really fun and cool cards and mechanic introduced the decks im currently working on are chaos cyber dark and crystal beast none of them ha', 've the same spark as ehero though any recommendations thanks 3 ive never seen these before very cute ill look into it i love the purple haired character in unchained definitely gonna look into this thanks f', 'or the recommendation also swordsouls chinese theme is fire might make this my synchro deck i saw the art from phantom knights earlier today and fell in love 

In [11]:
user_ids = ground_truth_df['user'].values

random.shuffle(user_ids)

split_idx = int(0.9 * len(user_ids))
train_users = user_ids[:split_idx]
test_users  = user_ids[split_idx:]

len(train_users), len(test_users)

(818, 91)

In [12]:
def build_examples(selected_users):
    texts, labels, users = [], [], []
    for uid in selected_users:
        segments = processed_posts[uid]
        label = ground_truth_df[ground_truth_df['user'] == uid]['is_depressed'].values[0]
        for seg in segments:
            texts.append(seg)
            labels.append(label)
            users.append(uid)
    return texts, labels, users

In [13]:
train_texts, train_labels, train_user = build_examples(train_users)

len(train_texts), len(train_labels), len(train_user)

(28000, 28000, 28000)

In [14]:
train_texts[0], train_labels[0], train_user[0]

('what about an apk file you will need to install an apk installer app first i used this website to put it on a friends tablet this one is in my experience i trust this site and apkmody nothing else i also have a threat detector in my device and it says this site is safe you download a package installer app and from there you can install the apk to your device theres lots of good ones on google play just make sure it says its capable of installing apk files same i usually wait untill they sit down at their seats in the morning and put their bags down this time i was setting things up and forgot to press the stop time lapse button haha great lol yes once i saw one flopping around in the roadcrazy fish what this is called is negative coloring for every color there is a negative color and when you look at her for 20 seconds then look at a wall you should see her in her normal colors if you were to stare at a image in its usual colors positive then look at the wall you should see the negat

In [15]:
test_texts, test_labels, test_user = build_examples(test_users)
len(test_texts), len(test_labels), len(test_user)

(3115, 3115, 3115)

In [16]:
test_texts[0], test_labels[0], test_user[0]

('get in the ufo shinji honestly sounds more like a brilliant strategy no ones there at that time great is a strong word brother from asukas smile tough love really is the best because why wouldnt we want her rlostredditors i dont like the guy either the devils spawn as a floridian i approve of this message ai eh still ai art is the death of art good bot ive seen so much nikado avocado in the past 24 hours i cant escape him the mukbang youtuber nikado avocado went through a two year long weight loss journey and pulled the rug on everyone the funny is that shinji is the sub this sub is slowly turning into a jet alone circle jerk though can i have the song the british were favoring the arabs in then mandate of palestine and many zionists especially revisionists then even considered violence to the british and there was never a palestinian state before the british mandate of palestine it was occupied by the ottoman empire as southern syria the more you go back the only palestine youll fin

In [17]:
train_dataset = Dataset.from_dict({
    "text": train_texts,
    "label": train_labels,
    "user": train_user
})
test_dataset = Dataset.from_dict({
    "text": test_texts,
    "label": test_labels,
    "user": test_user
})

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 28000
    })
    test: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 3115
    })
})

In [18]:
split = dataset["train"].train_test_split(test_size=0.1, seed=42)

dataset = DatasetDict({
    "train": split["train"],
    "val": split["test"],
    "test": dataset["test"]
})

dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 25200
    })
    val: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 2800
    })
    test: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 3115
    })
})

## Loading models

In [19]:
model_path = '/kaggle/input/disorbert-domain-adapted/pytorch/default/1/bert-base-uncased-domain-adapted-mental-dataset'

config = AutoConfig.from_pretrained(model_path)
config.num_labels = 2

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    config=config,
)
print(model.classifier)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/disorbert-domain-adapted/pytorch/default/1/bert-base-uncased-domain-adapted-mental-dataset and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=2, bias=True)


In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [21]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [22]:
tokenizer

BertTokenizerFast(name_or_path='/kaggle/input/disorbert-domain-adapted/pytorch/default/1/bert-base-uncased-domain-adapted-mental-dataset', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [23]:
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

tokenized_dataset = dataset.map(tokenize_fn, batched=True)
tokenized_dataset

Map:   0%|          | 0/25200 [00:00<?, ? examples/s]

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3115 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25200
    })
    val: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2800
    })
    test: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3115
    })
})

In [24]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds),
        'recall': recall_score(labels, preds),
        'f1': f1_score(labels, preds)
    }

In [25]:
args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    save_total_limit=2,
    eval_strategy="epoch", 
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=30,
    weight_decay=0.01,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipykernel_19/3303828671.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.342100,0.316923,0.892857,0.522222,0.154605,0.238579
2,0.242700,0.305520,0.891429,0.500000,0.302632,0.377049
3,0.166500,0.430672,0.852143,0.350543,0.424342,0.383929
4,0.085700,0.552629,0.893571,0.518072,0.282895,0.365957
5,0.056600,0.575465,0.901071,0.597122,0.273026,0.374718
6,0.032800,0.691196,0.882143,0.445833,0.351974,0.393382
7,0.025700,0.765976,0.903214,0.685393,0.200658,0.310433
8,0.025200,0.817367,0.894643,0.531469,0.250000,0.340045
9,0.016000,0.888230,0.903214,0.643478,0.243421,0.353222
10,0.015200,0.791786,0.897143,0.553333,0.273026,0.365639


TrainOutput(global_step=23640, training_loss=0.03587611788937384, metrics={'train_runtime': 8832.445, 'train_samples_per_second': 85.594, 'train_steps_per_second': 2.676, 'total_flos': 4.972798946304e+16, 'train_loss': 0.03587611788937384, 'epoch': 30.0})

## Testing

In [26]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

predictions = trainer.predict(tokenized_dataset["test"])

segment_preds = np.argmax(predictions.predictions, axis=1)
segment_labels = predictions.label_ids
user_ids = tokenized_dataset["test"]["user"]

df = pd.DataFrame({
    "user_id": user_ids,
    "segment_pred": segment_preds,
    "segment_label": segment_labels
})

def majority_vote(group):
    return int(group["segment_pred"].mean() >= 0.5)

user_preds = df.groupby("user_id", group_keys=False).apply(majority_vote)
user_labels = df.groupby("user_id")["segment_label"].first()

accuracy = accuracy_score(user_labels, user_preds)
precision = precision_score(user_labels, user_preds, zero_division=1)
recall = recall_score(user_labels, user_preds, zero_division=1)
f1 = f1_score(user_labels, user_preds, zero_division=1)

print(f"✅ User-level Results:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

results_df = pd.DataFrame({
    "user_id": user_labels.index,
    "true_label": user_labels.values,
    "pred_label": user_preds.values
})
results_df.to_csv("user_level_results.csv", index=False)

✅ User-level Results:
Accuracy:  0.8989
Precision: 1.0000
Recall:    0.0000
F1 Score:  0.0000


/tmp/ipykernel_19/1608300634.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_preds = df.groupby("user_id", group_keys=False).apply(majority_vote)
